#### Model Selection

In [1]:
import pickle
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, recall_score, precision_score, mean_squared_error,f1_score

In [2]:
# Loading the pickle file Customer_Loan

with open('Pickle_Files/Mutual_Fund.pkl', 'rb') as f1:
     df_train_mf= pickle.load(f1)
with open('Pickle_Files/40pct.pkl', 'rb') as f2:
     df_40= pickle.load(f2)

##### Train Test Split

In [3]:
# Product - Sale
X_mf = df_train_mf.copy()
X_mf.drop(['Revenue_MF','Sale_MF'],inplace=True,axis=1)

y_sale_mf = df_train_mf['Sale_MF']

X_train_sl,X_test_sl,y_train_sl,y_test_sl = train_test_split(X_mf,y_sale_mf,test_size=0.2,random_state=0)

In [4]:
X_mf.shape

(969, 28)

In [5]:
X_train_sl.head()

,Client,Sex,Age,Tenure,Count_CA,Count_SA,Count_OVD,Count_CC,Count_CL,ActBal_CA,...,VolumeDeb,VolumeDeb_CA,VolumeDebCash_Card,VolumeDebCashless_Card,VolumeDeb_PaymentOrder,TransactionsDeb,TransactionsDeb_CA,TransactionsDebCash_Card,TransactionsDebCashless_Card,TransactionsDeb_PaymentOrder
311,735,1,23,53,1,0.0,0.0,0.0,0.0,188.994643,...,339.935714,339.935714,214.285714,104.757143,16.500000,20.0,20.0,6.0,8.0,1.0
773,442,1,13,38,1,0.0,1.0,0.0,0.0,1083.015357,...,1739.750000,1739.750000,1178.571429,243.535714,311.392857,14.0,14.0,3.0,1.0,6.0
1347,955,0,19,62,1,0.0,0.0,0.0,0.0,3131.223929,...,246.528571,246.528571,0.000000,0.000000,129.642857,6.0,6.0,0.0,0.0,3.0
99,469,1,38,183,1,0.0,0.0,0.0,0.0,4415.646071,...,1549.925000,1549.925000,500.000000,0.000000,789.928571,13.0,13.0,1.0,0.0,7.0
128,536,1,29,14,1,0.0,0.0,0.0,0.0,663.843571,...,185.642857,185.642857,0.000000,0.000000,183.464286,2.0,2.0,0.0,0.0,1.0


In [6]:
# Revenue
X_mf = df_train_mf.copy()
X_mf.drop(['Revenue_MF','Sale_MF'],inplace=True,axis=1)

y_revenue_mf = df_train_mf['Revenue_MF']

X_train_rev,X_test_rev,y_train_rev,y_test_rev = train_test_split(X_mf,y_revenue_mf,test_size=0.2,random_state=0)

##### Normalize The Features

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train_sl)
X_test = scaler.transform(X_test_sl)

#### Cross Validation

##### Classification

In [8]:
model_param = {
    'KNeighbors': {
        'model': KNeighborsClassifier(),
        'param' : {'n_neighbors':[3,5,10,100],
                   'weights':['uniform','distance']
                   }
        },
    'Logistic':{
        'model':LogisticRegression(max_iter=10000,solver='sag'), # converge warning hence sag
        'param': { 'C':[1,5,10]}
        },
    'RandomForest': {
        'model': RandomForestClassifier(),
        'param' : {'n_estimators':[5,10,50,100]}
        },
    'DecisionTree':{
        'model': DecisionTreeClassifier(),
        'param' : {'criterion': ['gini', 'entropy'],
            'max_depth': [None, 10, 20, 30],  
            'min_samples_split': [2, 5, 10],  
            'min_samples_leaf': [1, 2, 4]}
        }
}

In [9]:
scores_class = []

for model_name, mp in model_param.items():
    hyperpara_model = GridSearchCV(mp['model'],mp['param'],cv=5,return_train_score=False)
    hyperpara_model.fit(X_mf,y_sale_mf)
    scores_class.append({
        'model': model_name,
        'best_score':hyperpara_model.best_score_,
        'best_param':hyperpara_model.best_params_
    })

In [10]:
scores_mf = pd.DataFrame(scores_class)
scores_mf

,model,best_score,best_param
0,KNeighbors,0.809075,"{'n_neighbors': 10, 'weights': 'uniform'}"
1,Logistic,0.804941,{'C': 1}
2,RandomForest,0.803926,{'n_estimators': 50}
3,DecisionTree,0.765750,"{'criterion': 'entropy', 'max_depth': 10, 'min..."


RandomForest seems to have the best score. Hence, I will be using this for model prediction. It is also more robust towards imbalanced datasets compared to KNeighbors + Logistic + DecisionTrees, as they are more biased towards majority data

In [11]:
rf = RandomForestClassifier(n_estimators=50)

rf.fit(X_train_sl,y_train_sl)

RandomForestClassifier(n_estimators=50)

In [12]:
y_pred_sl = rf.predict(X_test_sl)

##### Evaluation

In [13]:
confusion_matrix(y_pred_sl,y_test_sl)

array([[149,  42],
       [  1,   2]])

In [14]:
print('Evaluation metrics:')
print('Accuracy:', accuracy_score(y_test_sl,y_pred_sl))
print('Precision:', precision_score(y_test_sl, y_pred_sl))
print('Recall:', recall_score(y_test_sl, y_pred_sl))
print('F-1:', f1_score(y_test_sl, y_pred_sl))
print('AUC:', roc_auc_score(y_test_sl, y_pred_sl))

Evaluation metrics:
Accuracy: 0.7783505154639175
Precision: 0.6666666666666666
Recall: 0.045454545454545456
F-1: 0.08510638297872342
AUC: 0.5193939393939393


##### Regression

In [13]:
# Define models
models = {
    'Linear': LinearRegression(),
    'RandomForest': RandomForestRegressor(),
    'DecisionTree': DecisionTreeRegressor(),
}

scores_rev = []

for model_name, model in models.items():
    # Use cross_val_score for simplicity
    cv_scores = cross_val_score(model, X_mf, y_revenue_mf, cv=10)
    scores_rev.append({
        'model': model_name,
        'mean_score': cv_scores.mean() * -1,
        'std_score': cv_scores.std(),
        
    })


In [14]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

scores_rev = pd.DataFrame(scores_rev)
scores_rev

,model,mean_score,std_score
0,Linear,0.540409,0.548479
1,RandomForest,0.914890,0.821164
2,DecisionTree,9.416892,14.815445


I will choose RandomForest as it has a good balance between accuracy and stability

In [15]:
rf = RandomForestRegressor(n_estimators=50)

rf.fit(X_train_rev,y_train_rev)

RandomForestRegressor(n_estimators=50)

In [16]:
y_pred_rev = rf.predict(X_test_rev)

##### Evaluation

In [17]:
mse=mean_squared_error(y_test_rev,y_pred_rev)
rmse=np.sqrt(mse)

print('MSE:', mse)
print('RMSE:', rmse)


MSE: 67.35611927454364
RMSE: 8.2070773893356


#### Targeting

In [18]:
df_40_mf = df_40.drop(['Count_MF','ActBal_MF','Revenue_CL','Sale_CL','Sale_CC','Sale_MF','Revenue_CC','Revenue_MF'], axis=1)

df_40_mf.shape

(646, 28)

In [19]:
scaler.transform(df_40_mf)

array([[ 0.20291412,  0.93891075, -1.18385804, ..., -0.36391167,
        -0.63191716, -0.10413506],
       [-0.24935203,  0.93891075, -0.32222368, ...,  0.00688792,
        -0.63191716,  0.08733908],
       [-0.61762589, -1.06506396, -1.29156234, ...,  0.74848709,
        -0.27158307, -0.48708335],
       ...,
       [-0.75545938,  0.93891075,  1.34719288, ...,  0.00688792,
        -0.27158307,  0.27881323],
       [-0.87391004,  0.93891075, -0.64533657, ..., -0.36391167,
        -0.5118058 ,  0.85323566],
       [ 1.40249624,  0.93891075,  1.07793215, ..., -0.36391167,
        -0.5118058 , -0.2956092 ]])

In [20]:

rev_40=rf.predict(df_40_mf)

features_df = pd.DataFrame({
    'Client':  df_40['Client'].values,
    'Revenue_MF':  rev_40
}).sort_values('Revenue_MF', ascending= False)

features_df.head(10)


,Client,Revenue_MF
52,19,45.283346
55,10,44.448486
587,532,23.570839
269,505,22.732296
570,5,22.592100
466,16,22.588275
12,3,21.982064
287,766,21.186411
506,330,19.764989
175,24,18.811293


### Model Peformance:

Based on the machine learning model of random forest, the model's performance is ~52%. Moreover, the model has a precision of 1 and thus when divided by the 0.5 prevalence, would give 200%. Hence, the model does not perform better than guessing randomly. The model was only able to catch ~4% of clients who would buy mutual fund product.